**Criar o diretórios "tables" para receber os arquivos origem dos dados**

In [0]:
# Criar o diretório no Databricks
dbutils.fs.mkdirs("/FileStore/tables/")
print("Diretório criado com sucesso!")


In [0]:
import requests

# Links diretos dos arquivos CSV no GitHub
file_urls = {
    "orders": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_orders_dataset.csv",
    "customers": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_customers_dataset.csv",
    "products": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_products_dataset.csv",
    "ord_prds": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_items_dataset.csv",
    "categories": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/product_category_name_translation.csv",
    "sellers": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_sellers_dataset.csv",
    "payments": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_payments_dataset.csv",
    "ord_reviews": "https://raw.githubusercontent.com/gaedesig/MVP_2/refs/heads/main/olist_order_reviews_dataset.csv"
}

# Caminhos locais no Databricks para salvar os arquivos
local_paths = {name: f"/FileStore/tables/{name}.csv" for name in file_urls.keys()}

# Baixar cada arquivo e salvar no Databricks usando dbutils.fs.put
for name, url in file_urls.items():
    response = requests.get(url)
    if response.status_code == 200:  # Checar se o download foi bem-sucedido
        file_content = response.content.decode("utf-8")  # Decodificar o conteúdo do arquivo
        dbutils.fs.put(local_paths[name], file_content, overwrite=True)
        print(f"Arquivo {name} salvo em: {local_paths[name]}")
    else:
        print(f"Erro ao baixar o arquivo {name}: {response.status_code}")


Wrote 17654914 bytes.
Arquivo orders salvo em: /FileStore/tables/orders.csv
Wrote 9033957 bytes.
Arquivo customers salvo em: /FileStore/tables/customers.csv
Wrote 2379446 bytes.
Arquivo products salvo em: /FileStore/tables/products.csv
Wrote 15438671 bytes.
Arquivo ord_prds salvo em: /FileStore/tables/ord_prds.csv
Wrote 2613 bytes.
Arquivo categories salvo em: /FileStore/tables/categories.csv
Wrote 174703 bytes.
Arquivo sellers salvo em: /FileStore/tables/sellers.csv
Wrote 5777138 bytes.
Arquivo payments salvo em: /FileStore/tables/payments.csv
Wrote 14451670 bytes.
Arquivo ord_reviews salvo em: /FileStore/tables/ord_reviews.csv


In [0]:
# Listar os arquivos no diretório
dbutils.fs.ls("/FileStore/tables/")


Out[2]: [FileInfo(path='dbfs:/FileStore/tables/categories.csv', name='categories.csv', size=2613, modificationTime=1744328101000),
 FileInfo(path='dbfs:/FileStore/tables/customers.csv', name='customers.csv', size=9033957, modificationTime=1744328097000),
 FileInfo(path='dbfs:/FileStore/tables/ord_prds.csv', name='ord_prds.csv', size=15438671, modificationTime=1744328101000),
 FileInfo(path='dbfs:/FileStore/tables/ord_reviews.csv', name='ord_reviews.csv', size=14451670, modificationTime=1744328106000),
 FileInfo(path='dbfs:/FileStore/tables/orders.csv', name='orders.csv', size=17654914, modificationTime=1744328094000),
 FileInfo(path='dbfs:/FileStore/tables/payments.csv', name='payments.csv', size=5777138, modificationTime=1744328103000),
 FileInfo(path='dbfs:/FileStore/tables/products.csv', name='products.csv', size=2379446, modificationTime=1744328098000),
 FileInfo(path='dbfs:/FileStore/tables/sellers.csv', name='sellers.csv', size=174703, modificationTime=1744328102000)]

# Análise inicial dos dados

In [0]:

# Ler os arquivos CSV salvos

customers_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/customers.csv")
products_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/products.csv")
order_items_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/ord_prds.csv")
category_translation_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/categories.csv")
sellers_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/sellers.csv")
payments_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/payments.csv")
order_reviews_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/ord_reviews.csv")
orders_df = spark.read.format("csv").option("header", "true").load("/FileStore/tables/orders.csv")

# Criando dataframes em pandas para melhor visualização


In [0]:
import pandas as pd

In [0]:
# Converter o DataFrame Spark para Pandas
orders_pandas_df = orders_df.toPandas()
customer_pandas_df = customers_df.toPandas()
order_items_pandas_df = order_items_df.toPandas()
order_reviews_pandas_df = order_reviews_df.toPandas()
payments_pandas_df = payments_df.toPandas()
products_pandas_df = products_df.toPandas()
sellers_pandas_df = sellers_df.toPandas()
category_translation_pandas_df = category_translation_df.toPandas()



# Análise inicial nos dataframes que originarão as tabelas

Analisar nulos, valores ambíguos,  utilizando info(), describe() e outros
(ex hipotético: Ilhas das Flores <> I.Flores), possíveis outliers, etc


In [0]:
orders_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [0]:
orders_pandas_df.isnull().sum()

Out[7]: order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

## order_pandas_df
Contém nulos em 3 colunas. Entendo que esses valores nulos ñ devem influenciar.

A coluna que, provavelmente mais influenciaria, 'order_approved_at' tem
poucos valores nulos - 160 em mais de 99 mil registros

In [0]:
customer_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  object
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: object(5)
memory usage: 3.8+ MB


## customer_pandas_df
Não contém nenhuma coluna com valores nulos

In [0]:
payments_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   order_id              103886 non-null  object
 1   payment_sequential    103886 non-null  object
 2   payment_type          103886 non-null  object
 3   payment_installments  103886 non-null  object
 4   payment_value         103886 non-null  object
dtypes: object(5)
memory usage: 4.0+ MB


## payment_pandas_df 
Não contém nenhuma coluna com valores nulos

Achei valores max(999,68) e min(0,00) questionáveis. Vou avaliar na camada Gold em SQL

In [0]:
order_items_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_id             112650 non-null  object
 1   order_item_id        112650 non-null  object
 2   product_id           112650 non-null  object
 3   seller_id            112650 non-null  object
 4   shipping_limit_date  112650 non-null  object
 5   price                112650 non-null  object
 6   freight_value        112650 non-null  object
dtypes: object(7)
memory usage: 6.0+ MB


## order_items_pandas_df
Não contém nenhuma coluna com valores nulos

Útil na camada gold para análise

In [0]:
order_reviews_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104162 entries, 0 to 104161
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   review_id                104161 non-null  object
 1   order_id                 101926 non-null  object
 2   review_score             101782 non-null  object
 3   review_comment_title     12005 non-null   object
 4   review_comment_message   41083 non-null   object
 5   review_creation_date     95398 non-null   object
 6   review_answer_timestamp  95377 non-null   object
dtypes: object(7)
memory usage: 5.6+ MB


In [0]:
order_reviews_pandas_df.isnull().sum()

Out[17]: review_id                      1
order_id                    2236
review_score                2380
review_comment_title       92157
review_comment_message     63079
review_creation_date        8764
review_answer_timestamp     8785
dtype: int64

In [0]:
order_reviews_pandas_df.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


## order_reviews_pandas_df
Algumas colunas contêm muitos valores nulos. Avaliar melhor a importância disso.

Entendo porém, que as principais como, review_score contêm poucos

In [0]:
category_translation_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 2 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   product_category_name          71 non-null     object
 1   product_category_name_english  71 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB


In [0]:
# Unique
pd.unique(category_translation_pandas_df['product_category_name'])

## category_translation_pandas_df
Não contém nenhuma coluna com valores nulos, nem valores ambíguos

In [0]:
products_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   product_id                  32951 non-null  object
 1   product_category_name       32341 non-null  object
 2   product_name_lenght         32341 non-null  object
 3   product_description_lenght  32341 non-null  object
 4   product_photos_qty          32341 non-null  object
 5   product_weight_g            32949 non-null  object
 6   product_length_cm           32949 non-null  object
 7   product_height_cm           32949 non-null  object
 8   product_width_cm            32949 non-null  object
dtypes: object(9)
memory usage: 2.3+ MB


In [0]:
products_pandas_df.isnull().sum()

Out[22]: product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64

## products_pandas_df
8 colunas em 9 contêm valores nulos.

Um ponto negativo é que esse dataframe não contém a descrição do produto, ficando dependendo muito da category_name, que contém centenas de 
valores nulos.

In [0]:
sellers_pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   object
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: object(4)
memory usage: 96.8+ KB


In [0]:
pd.unique(sellers_pandas_df['seller_city'])

## sellers_pandas_df
Não contém colunas com valores nulos

Como em categorias, valores devem ser escolhidos de uma lista no site

%md
# Fim da análise inicial

## Criando a camada Bronze

In [0]:
# Criando o diretório "bronze" no Databricks
bronze_path = "/mnt/bronze/"    # Diretório para persistir os dados na camada Bronze
dbutils.fs.mkdirs(bronze_path)  # Criar o diretório no Databricks


Out[21]: True

In [0]:
# Persistindo os dataframes na camada "bronze"

# Persistir o DataFrame orders_df
orders_df.write.format("delta").mode("overwrite").save("/mnt/bronze/orders")

# Persistir o DataFrame customers_df
customers_df.write.format("delta").mode("overwrite").save("/mnt/bronze/customers")

# Persistir o DataFrame products_df
products_df.write.format("delta").mode("overwrite").save("/mnt/bronze/products")


# Persistir o DataFrame order_items_df
order_items_df.write.format("delta").mode("overwrite").save("/mnt/bronze/order_items")


# Persistir o DataFrame category_translation_df
category_translation_df.write.format("delta").mode("overwrite").save("/mnt/bronze/category_translation")


# Persistir o DataFrame sellers_df
sellers_df.write.format("delta").mode("overwrite").save("/mnt/bronze/sellers")


# Persistir o DataFrame payments_df
payments_df.write.format("delta").mode("overwrite").save("/mnt/bronze/payments")


# Persistir o DataFrame order_reviews_df
order_reviews_df.write.format("delta").mode("overwrite").save("/mnt/bronze/ord_reviews")


print("DataFrames persistidos com sucesso na camada Bronze!")


DataFrames persistidos com sucesso na camada Bronze!


In [0]:
# Listar os arquivos no diretório "bronze"
dbutils.fs.ls("/mnt/bronze/")


Out[24]: [FileInfo(path='dbfs:/mnt/bronze/category_translation/', name='category_translation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/customers/', name='customers/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/ord_reviews/', name='ord_reviews/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/order_items/', name='order_items/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/products/', name='products/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/bronze/sellers/', name='sellers/', size=0, modificationTime=0)]

In [0]:
# Carregar dados da camada Bronze criando dataframes
bronze_orders_df = spark.read.format("delta").load("/mnt/bronze/orders")
bronze_customers_df = spark.read.format("delta").load("/mnt/bronze/customers")
bronze_products_df = spark.read.format("delta").load("/mnt/bronze/products")
bronze_order_items_df = spark.read.format("delta").load("/mnt/bronze/order_items")
bronze_category_translation_df = spark.read.format("delta").load("/mnt/bronze/category_translation")
bronze_sellers_df = spark.read.format("delta").load("/mnt/bronze/sellers")
bronze_payments_df = spark.read.format("delta").load("/mnt/bronze/payments")
bronze_order_reviews = spark.read.format("delta").load("/mnt/bronze/ord_reviews")



In [0]:
# Registrar os DataFrames como Temporary Views
bronze_orders_df.createOrReplaceTempView("bronze_orders")
bronze_customers_df.createOrReplaceTempView("bronze_customers")
bronze_products_df.createOrReplaceTempView("bronze_products")
bronze_order_items_df.createOrReplaceTempView("bronze_order_items")
bronze_category_translation_df.createOrReplaceTempView("bronze_category_translation")
bronze_sellers_df.createOrReplaceTempView("bronze_sellers")
bronze_payments_df.createOrReplaceTempView("bronze_payments")
bronze_order_reviews.createOrReplaceTempView("bronze_order_reviews")
print("Views temporárias criadas com sucesso!")


Views temporárias criadas com sucesso!


In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
,bronze_category_translation,true
,bronze_customers,true
,bronze_order_items,true
,bronze_order_reviews,true
,bronze_orders,true
,bronze_payments,true
,bronze_products,true
,bronze_sellers,true


## 8 Views criadas

**Criar tabelas com essas views**


In [0]:
%sql
-- Criar tabela Delta para bronze_orders
CREATE TABLE delta_bronze_orders
USING DELTA
AS SELECT * FROM bronze_orders;

-- Criar tabela Delta para bronze_customers
CREATE TABLE delta_bronze_customers
USING DELTA
AS SELECT * FROM bronze_customers;

-- Criar tabela Delta para bronze_products
CREATE TABLE delta_bronze_products
USING DELTA
AS SELECT * FROM bronze_products;

-- Criar tabela Delta para bronze_order_items
CREATE TABLE delta_bronze_order_items
USING DELTA
AS SELECT * FROM bronze_order_items;

-- Criar tabela Delta para bronze_category_translation
CREATE TABLE delta_bronze_category_translation
USING DELTA
AS SELECT * FROM bronze_category_translation;

-- Criar tabela Delta para bronze_sellers
CREATE TABLE delta_bronze_sellers
USING DELTA
AS SELECT * FROM bronze_sellers;

-- Criar tabela Delta para bronze_payments
CREATE TABLE delta_bronze_payments
USING DELTA
AS SELECT * FROM bronze_payments;

-- Criar tabela Delta para bronze_order_reviews
CREATE TABLE delta_bronze_order_reviews
USING DELTA
AS SELECT * FROM bronze_order_reviews;

-- Exibir confirmação
SHOW TABLES;


## Passando para a camada prata

In [0]:
# Diretório da camada Prata
silver_path = "/mnt/silver/"
dbutils.fs.mkdirs(silver_path)

Out[27]: True

In [0]:
# Listar os arquivos no diretório Silver
dbutils.fs.ls("/mnt/silver/")


Out[38]: [FileInfo(path='dbfs:/mnt/silver/category_translation/', name='category_translation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/customers/', name='customers/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/order_items/', name='order_items/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/order_reviews/', name='order_reviews/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/products/', name='products/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/silver/sellers/', name='sellers/', size=0, modificationTime=0)]

In [0]:
# Copiar DataFrames diretamente para a camada Prata 
bronze_orders_df.write.format("delta").mode("overwrite").save(f"{silver_path}orders")
bronze_customers_df.write.format("delta").mode("overwrite").save(f"{silver_path}customers")
bronze_products_df.write.format("delta").mode("overwrite").save(f"{silver_path}products")
bronze_order_items_df.write.format("delta").mode("overwrite").save(f"{silver_path}order_items")
bronze_category_translation_df.write.format("delta").mode("overwrite").save(f"{silver_path}category_translation")
bronze_sellers_df.write.format("delta").mode("overwrite").save(f"{silver_path}sellers")
bronze_payments_df.write.format("delta").mode("overwrite").save(f"{silver_path}payments")
bronze_order_reviews.write.format("delta").mode("overwrite").save(f"{silver_path}order_reviews")

In [0]:
# Carregar dados da camada Bronze criando dataframes
silver_orders_df = spark.read.format("delta").load("/mnt/silver/orders")
silver_customers_df = spark.read.format("delta").load("/mnt/silver/customers")
silver_products_df = spark.read.format("delta").load("/mnt/silver/products")
silver_order_items_df = spark.read.format("delta").load("/mnt/silver/order_items")
silver_category_translation_df = spark.read.format("delta").load("/mnt/silver/category_translation")
silver_sellers_df = spark.read.format("delta").load("/mnt/silver/sellers")
silver_payments_df = spark.read.format("delta").load("/mnt/silver/payments")
silver_order_reviews = spark.read.format("delta").load("/mnt/silver/order_reviews")



In [0]:
silver_orders_df.show()

+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|            order_id|         customer_id|order_status|order_purchase_timestamp|  order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------------------+--------------------+------------+------------------------+-------------------+----------------------------+-----------------------------+-----------------------------+
|e481f51cbdc54678b...|9ef432eb625129730...|   delivered|     2017-10-02 10:56:33|2017-10-02 11:07:15|         2017-10-04 19:55:00|          2017-10-10 21:25:13|          2017-10-18 00:00:00|
|53cdb2fc8bc7dce0b...|b0830fb4747a6c6d2...|   delivered|     2018-07-24 20:41:37|2018-07-26 03:24:27|         2018-07-26 14:31:00|          2018-08-07 15:27:45|          2018-08-13 00:00:00|
|47770eb9100c2d0c4...|41ce2a54c0b03bf34...|  

In [0]:
# Registrar os DataFrames como Temporary Views
silver_orders_df.createOrReplaceTempView("silver_orders")
silver_customers_df.createOrReplaceTempView("silver_customers")
silver_products_df.createOrReplaceTempView("silver_products")
silver_order_items_df.createOrReplaceTempView("silver_order_items")
silver_category_translation_df.createOrReplaceTempView("silver_category_translation")
silver_sellers_df.createOrReplaceTempView("silver_sellers")
silver_payments_df.createOrReplaceTempView("silver_payments")
silver_order_reviews.createOrReplaceTempView("silver_order_reviews")
print("Views temporárias criadas com sucesso!")


Views temporárias criadas com sucesso!


In [0]:
%sql
SHOW TABLES

database,tableName,isTemporary
default,delta_bronze_category_translation,false
default,delta_bronze_customers,false
default,delta_bronze_order_items,false
default,delta_bronze_order_reviews,false
default,delta_bronze_orders,false
default,delta_bronze_payments,false
default,delta_bronze_products,false
default,delta_bronze_sellers,false
,bronze_category_translation,true
,bronze_customers,true


In [0]:
%sql
-- Criar tabela Delta para silver_orders
CREATE TABLE delta_silver_orders
USING DELTA
AS SELECT * FROM silver_orders;

-- Criar tabela Delta para silver_customers
CREATE TABLE delta_silver_customers
USING DELTA
AS SELECT * FROM silver_customers;

-- Criar tabela Delta para silver_products
CREATE TABLE delta_silver_products
USING DELTA
AS SELECT * FROM silver_products;

-- Criar tabela Delta para silver_order_items
CREATE TABLE delta_silver_order_items
USING DELTA
AS SELECT * FROM silver_order_items;

-- Criar tabela Delta para silver_category_translation
CREATE TABLE delta_silver_category_translation
USING DELTA
AS SELECT * FROM silver_category_translation;

-- Criar tabela Delta para silver_sellers
CREATE TABLE delta_silver_sellers
USING DELTA
AS SELECT * FROM silver_sellers;

-- Criar tabela Delta para silver_payments
CREATE TABLE delta_silver_payments
USING DELTA
AS SELECT * FROM silver_payments;

-- Criar tabela Delta para silver_order_reviews
CREATE TABLE delta_silver_order_reviews
USING DELTA
AS SELECT * FROM silver_order_reviews;

-- Exibir confirmação
SHOW TABLES;


database,tableName,isTemporary
default,delta_bronze_category_translation,false
default,delta_bronze_customers,false
default,delta_bronze_order_items,false
default,delta_bronze_order_reviews,false
default,delta_bronze_orders,false
default,delta_bronze_payments,false
default,delta_bronze_products,false
default,delta_bronze_sellers,false
default,delta_silver_category_translation,false
default,delta_silver_customers,false


## Fim da camada prata

## Início camada gold

## Criando diretório

In [0]:
# Diretório da camada gold
gold_path = "/mnt/gold/"
dbutils.fs.mkdirs(gold_path)

Out[32]: True

In [0]:
# Copiar DataFrames diretamente para a camada gold
silver_orders_df.write.format("delta").mode("overwrite").save(f"{gold_path}orders")
silver_customers_df.write.format("delta").mode("overwrite").save(f"{gold_path}customers")
silver_products_df.write.format("delta").mode("overwrite").save(f"{gold_path}products")
silver_order_items_df.write.format("delta").mode("overwrite").save(f"{gold_path}order_items")
silver_category_translation_df.write.format("delta").mode("overwrite").save(f"{gold_path}category_translation")
silver_sellers_df.write.format("delta").mode("overwrite").save(f"{gold_path}sellers")
silver_payments_df.write.format("delta").mode("overwrite").save(f"{gold_path}payments")
silver_order_reviews.write.format("delta").mode("overwrite").save(f"{gold_path}order_reviews")

In [0]:
# Listar os arquivos no diretório gold
dbutils.fs.ls("/mnt/gold/")


Out[42]: [FileInfo(path='dbfs:/mnt/gold/category_translation/', name='category_translation/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/customers/', name='customers/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/order_items/', name='order_items/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/order_reviews/', name='order_reviews/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/orders/', name='orders/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/payments/', name='payments/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/products/', name='products/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/gold/sellers/', name='sellers/', size=0, modificationTime=0)]

In [0]:
# Criar dataframes gold_nome_df em "/mnt/gold/"


In [0]:
# Carregar dados da camada Bronze criando dataframes
gold_orders_df = spark.read.format("delta").load("/mnt/gold/orders")
gold_customers_df = spark.read.format("delta").load("/mnt/gold/customers")
gold_products_df = spark.read.format("delta").load("/mnt/gold/products")
gold_order_items_df = spark.read.format("delta").load("/mnt/gold/order_items")
gold_category_translation_df = spark.read.format("delta").load("/mnt/gold/category_translation")
gold_sellers_df = spark.read.format("delta").load("/mnt/gold/sellers")
gold_payments_df = spark.read.format("delta").load("/mnt/gold/payments")
gold_order_reviews = spark.read.format("delta").load("/mnt/gold/order_reviews")

In [0]:
# Criação de views
gold_orders_df.createOrReplaceTempView("gold_orders")
gold_customers_df.createOrReplaceTempView("gold_customers")
gold_products_df.createOrReplaceTempView("gold_products")
gold_order_items_df.createOrReplaceTempView("gold_order_items")
gold_category_translation_df.createOrReplaceTempView("gold_category_translation")
gold_sellers_df.createOrReplaceTempView("gold_sellers")
gold_payments_df.createOrReplaceTempView("gold_payments")
gold_order_reviews.createOrReplaceTempView("gold_order_reviews")
print("Views gold temporárias criadas com sucesso!")

Views gold temporárias criadas com sucesso!


In [0]:
%sql
-- Criar tabela Delta para gold_orders
CREATE TABLE delta_gold_orders
USING DELTA
AS SELECT * FROM gold_orders;

-- Criar tabela Delta para gold_customers
CREATE TABLE delta_gold_customers
USING DELTA
AS SELECT * FROM gold_customers;

-- Criar tabela Delta para gold_products
CREATE TABLE delta_gold_products
USING DELTA
AS SELECT * FROM gold_products;

-- Criar tabela Delta para gold_order_items
CREATE TABLE delta_gold_order_items
USING DELTA
AS SELECT * FROM gold_order_items;

-- Criar tabela Delta para gold_category_translation
CREATE TABLE delta_gold_category_translation
USING DELTA
AS SELECT * FROM gold_category_translation;

-- Criar tabela Delta para gold_sellers
CREATE TABLE delta_gold_sellers
USING DELTA
AS SELECT * FROM gold_sellers;

-- Criar tabela Delta para gold_payments
CREATE TABLE delta_gold_payments
USING DELTA
AS SELECT * FROM gold_payments;

-- Criar tabela Delta para gold_order_reviews
CREATE TABLE delta_gold_order_reviews
USING DELTA
AS SELECT * FROM gold_order_reviews;

-- Exibir confirmação
SHOW TABLES;


%md
Informar no readme que, durante a execução do projeto, na análise dos dados, outras perguntas surgiram..

Analisar o mercado on-line de forma sintética (Produtos mais vendidos, Produtos que mais faturaram, faixa etária que mais compra, etc.) e analítica (vendas por região, produtos por faixa etária e região, formas de pagamento por faixa etária, por período, etc.).

In [0]:
# 1. Qual o produto mais vendido no período registrado?
#    a. Como estão divididas as faixas etárias na compra desse produto?
# Tabelas products, order e order_items


## Questão 1
**Qual o produto mais vendido no período registrado?**

In [0]:
%sql
-- Questão 1
SELECT oi.product_id, COUNT(oi.product_id) AS qtd_item, p.product_category_name
FROM delta_gold_order_items oi, delta_gold_products p
WHERE oi.product_id = p.product_id
GROUP BY oi.product_id, p.product_category_name
ORDER BY qtd_item DESC

**De acordo com a consulta acima, o produto mais vendido (em quantidade) no período, é da categoria móveis e decoraçãoes, com 527 unidades**


## Questão 1.a
**Como estão divididas as faixas etárias na compra desse produto?**

Os arquivos csv fonte não contêm qualquer tipo de dado que indique a idade do comprador. O arquivo de cadastro do comprador é muito simples, com poucas colunas e nenhuma delas sobre informações pessoais. Nem data de nascimento, nem sexo.


In [0]:
# 2. Qual a forma de pagamento mais usada nas transações?
#    a. Qual a faixe etária que mais utiliza esse método?
#    b. Qual o percentual de cada faixa etária que utilizam esse método?
# Tabelas order e payments

## Questão 2
**Qual a forma de pagamento mais usada nas transações?**

In [0]:
%sql
SELECT payment_type, COUNT(payment_type)
FROM delta_gold_payments
GROUP BY payment_type
ORDER BY COUNT(payment_type) DESC

payment_type,count(payment_type)
credit_card,76795
boleto,19784
voucher,5775
debit_card,1529
not_defined,3


A forma de pagamento mais usada nas transações é com cartão de crédito

In [0]:
%sql
SELECT
	P.payment_type,
	round(avg(P.payment_installments)) AVG_installments,
	round(avg(P.payment_value)) AVG_value,
	min(P.payment_value) MINimum,
	max(P.payment_value) MAXimum,
	PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY P.payment_value) AS median_payment_value
FROM
	delta_gold_payments P
WHERE 
	P.payment_value > 0
GROUP BY rollup
	(P.payment_type)
ORDER BY
	AVG_value desc;

payment_type,AVG_installments,AVG_value,MINimum,MAXimum,median_payment_value
credit_card,4.0,164.0,1.01,999.64,106.95
null,3.0,154.0,1.00,999.68,100.0
boleto,1.0,145.0,100.00,999.68,93.89
debit_card,1.0,143.0,100.19,99.90,89.3
voucher,1.0,67.0,1.00,99.98,40.11


## Questão 2.a
Como na questão 1.a, os arquivos csv fonte não contêm qualquer tipo de dado que indique a idade do comprador. O arquivo de cadastro do comprador é muito simples, com poucas colunas e nenhuma delas sobre informações pessoais. 

## Questão 2.b
Mesma limitação das questões 1.a e 2.a

In [0]:
# 3.Qual o percentual de cada categoria de produto nas vendas cada estado? 
# Tabelas products, orders, order_items


## Questão 3
**Qual o percentual de cada categoria de produto nas vendas cada estado?** 


In [0]:
%sql
WITH vendas AS (
  SELECT c.customer_state AS estado, p.product_category_name AS categoria, COUNT(oi.product_id) AS produtos
  FROM delta_gold_orders o LEFT JOIN delta_gold_order_items oi ON (oi.order_id = o.order_id)
                             LEFT JOIN delta_gold_products p ON (p.product_id = oi.product_id)
                             LEFT JOIN delta_gold_customers c ON (c.customer_id = o.customer_id)
  GROUP BY c.customer_state, p.product_category_name
  HAVING p.product_category_name IS NOT NULL                           
)
SELECT estado, categoria, produtos,
       SUM(produtos) OVER (PARTITION BY estado) AS total_vendas_estado,
       produtos*1.0/SUM(produtos) OVER (PARTITION BY estado) AS percentual_vendas
FROM vendas v 
ORDER BY estado, percentual_vendas DESC
-- Encontra o percentual de vendas de cada categoria de produtos em cada estado

estado,categoria,produtos,total_vendas_estado,percentual_vendas
AC,moveis_decoracao,12,90,0.1333333333333333
AC,esporte_lazer,9,90,0.1000000000000000
AC,informatica_acessorios,9,90,0.1000000000000000
AC,beleza_saude,7,90,0.0777777777777778
AC,telefonia,5,90,0.0555555555555556
AC,relogios_presentes,4,90,0.0444444444444444
AC,utilidades_domesticas,4,90,0.0444444444444444
AC,automotivo,4,90,0.0444444444444444
AC,cama_mesa_banho,4,90,0.0444444444444444
AC,eletronicos,4,90,0.0444444444444444


In [0]:
%sql
SELECT customer_state, COUNT(DISTINCT customer_id) AS qtd_clientes
FROM delta_gold_customers
GROUP BY customer_state
ORDER BY qtd_clientes DESC
-- QTD de clientes por estado em ordem decrescente

customer_state,qtd_clientes
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033
GO,2020


In [0]:
%sql
SELECT product_category_name, COUNT(DISTINCT product_id) AS qtd_produtos
FROM delta_silver_products p
GROUP BY product_category_name
ORDER BY qtd_produtos DESC
-- QTD de produtos de cada categoria

In [0]:
%sql
SELECT order_item_id, order_id, product_id, price FROM delta_bronze_order_items
ORDER BY price DESC
-- Ajustar para gold

In [0]:
# 4. Quais os vendedores com as melhores médias de avaliações?

In [0]:
%sql
WITH reviews AS 
(SELECT o.order_id, r.review_id, r.review_score FROM
delta_gold_order_reviews r JOIN delta_gold_orders o 
ON r.order_id=o.order_id),

order_details AS 
(SELECT i.seller_id, reviews.order_id, reviews.review_id, reviews.review_score FROM
reviews JOIN delta_gold_order_items i
ON reviews.order_id=i.order_id)

SELECT seller_id AS seller, COUNT(distinct review_id) AS total_reviews, 
ROUND(AVG(review_score),2) AS avg_ratings FROM order_details
GROUP BY seller_id
HAVING COUNT(distinct review_id)>200
ORDER BY avg_ratings DESC
LIMIT 10

seller,total_reviews,avg_ratings
c3cfdc648177fdbbbb35635a37472c53,282,4.45
fa40cc5b934574b62717c68f3d678b6d,306,4.43
7299e27ed73d2ad986de7f7c77d919fa,338,4.42
a3a38f4affed601eb87a97788c949667,250,4.39
fe2032dab1a61af8794248c8196565c9,290,4.38
87142160b41353c4e5fca2360caf6f92,307,4.37
6edacfd9f9074789dad6d62ba7950b9c,208,4.36
4b9750c8ad28220fe6702d4ecb7c898f,218,4.35
fa1c13f2614d7b5c4749cbc52fecda94,581,4.34
0ea22c1cfbdc755f86b9b54b39c16043,234,4.33


In [0]:
# 5. Qual o faturamento de cada categoria e seu percentual sobre o faturamento total no período?

## Questão 5
**Qual o faturamento de cada categoria e seu percentual sobre o faturamento total no período?**

In [0]:
%sql
WITH product_details AS
(SELECT product_id, product_category_name_english AS product_category
FROM delta_gold_products p join delta_gold_category_translation t
ON p.product_category_name=t.product_category_name),

order_details AS
(SELECT o.order_id, o.product_id, p.product_category, o.price
FROM product_details p join delta_gold_order_items o
ON p.product_id=o.product_id)

SELECT product_category, SUM(price) AS revenue, 100*SUM(price)/(SELECT SUM(price) FROM order_details) AS percent_revenue
FROM order_details 
GROUP BY product_category ORDER BY percent_revenue DESC
LIMIT 10

product_category,revenue,percent_revenue
health_beauty,1258681.339999968,9.38852437563066
watches_gifts,1205005.6800000058,8.988156763691865
bed_bath_table,1036988.6800000716,7.734915256179124
sports_leisure,988048.9700000364,7.369873171522781
computers_accessories,911954.320000041,6.802281952302785
furniture_decor,729762.4900000462,5.443310159652111
cool_stuff,635290.8500000037,4.738644676213527
housewares,632248.6600000246,4.715952932034572
auto,592720.110000013,4.421108841306689
garden_tools,485256.4600000161,3.6195357461504205


In [0]:
# 6. Qual faixa etária mais usa o cartão de crédito para pagar as compras?
# nenhum tipo de dado que possa fornecer essa informação. Cadastro de customers ñ tem

In [0]:
# 7. Quais os status dos pedidos entre janeiro de 2016 e dezembro de 2018?

## Questão 7
**Quais os status dos pedidos entre janeiro de 2016 e dezembro de 2018?**

In [0]:
%sql
SELECT order_status, COUNT(DISTINCT customer_id) AS qtd_clientes
FROM delta_gold_orders o 
WHERE( o.order_status IN ('processing','shipped','delivered')) AND
     (o.order_purchase_timestamp BETWEEN '2016-10-01' AND '2018-12-31')
GROUP BY o.order_status
HAVING qtd_clientes>1
-- Status das entregas dos pedidos em maio de 2016


order_status,qtd_clientes
shipped,1106
delivered,96477
processing,301


## Questão 8
**Quais os estados têm mais clientes registrados?**

In [0]:
%sql
SELECT customer_state, COUNT(DISTINCT customer_id) AS qtd_clientes
FROM delta_gold_customers
GROUP BY customer_state
ORDER BY qtd_clientes DESC
-- QTD de clientes por estado em ordem decrescente

customer_state,qtd_clientes
SP,41746
RJ,12852
MG,11635
RS,5466
PR,5045
SC,3637
BA,3380
DF,2140
ES,2033
GO,2020
